In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import the necessary packages
import os, sys
import torch
import torchvision
import time
import cv2

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch.nn as nn 
import torch.nn.functional as F
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
from datetime import datetime
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, datasets

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, Dropout, GlobalAveragePooling2D, MaxPooling2D
from tensorflow.keras import backend as K

In [3]:
# Import local helper files
from utils.dataset import load

In [4]:
# Check if CUDA is available
if torch.cuda.is_available():
    # Get the number of GPUs
    num_gpus = torch.cuda.device_count()
    print(f"Number of available CUDA devices: {num_gpus}")
    
    # Print the name of each GPU
    for i in range(num_gpus):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available.")

Number of available CUDA devices: 1
Device 0: NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [5]:
# Specify the shape of the inputs for our network
IMG_SIZE = 120
CROP_SIZE = 110
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

In [6]:
# Specify the batch size and number of epochs
BATCH_SIZE = 256
NUM_EPOCHS = 8
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Predefined Functions

In [7]:
# Predefined functions
# Function to load the dataset into Siamese Model
class FashionPairDataset(Dataset):
    def __init__(self, df, root_dir, transform=None):
        self.data_df = df
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.color_histograms = []
        self.edge_histograms = []
        self._load_images_and_features()

    def _load_images_and_features(self):
        for idx in range(len(self.data_df)):
            img_name = os.path.join(self.root_dir, self.data_df.iloc[idx, 0])
            image = Image.open(img_name)
            self.images.append(image)
            self.color_histograms.append(self._extract_color_histogram(image))
            self.edge_histograms.append(self._extract_edge_histogram(image))

    def _extract_color_histogram(self, image):
        image = np.array(image.convert("RGB"))
        hist = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
        cv2.normalize(hist, hist)
        return hist.flatten()

    def _extract_edge_histogram(self, image):
        image = np.array(image.convert("L"))
        edges = cv2.Canny(image, 100, 200)
        hist = cv2.calcHist([edges], [0], None, [256], [0, 256])
        cv2.normalize(hist, hist)
        return hist.flatten()
    
    def __len__(self):
        return len(self.data_df) * 2  # Each image will appear in one positive and one negative pair

    def __getitem__(self, idx):
        # Get the anchor image and its label
        anchor_idx = idx // 2
        anchor_image = self.images[anchor_idx]
        anchor_color_hist = self.color_histograms[anchor_idx]
        anchor_edge_hist = self.edge_histograms[anchor_idx]

        if self.transform:
            anchor_image = self.transform(anchor_image)
        
        if idx % 2 == 0:
            # Positive pair based on color similarity
            similar_idx = self._find_most_similar(anchor_idx, anchor_color_hist, self.color_histograms)
            similar_image = self.images[similar_idx]
            
            if self.transform:
                similar_image = self.transform(similar_image)
                
            return (anchor_image, similar_image), 1
        else:
            # Negative pair based on edge dissimilarity
            dissimilar_idx = self._find_most_dissimilar(anchor_idx, anchor_edge_hist, self.edge_histograms)
            dissimilar_image = self.images[dissimilar_idx]
            
            if self.transform:
                dissimilar_image = self.transform(dissimilar_image)
                
            return (anchor_image, dissimilar_image), 0
    
    def _find_most_similar(self, anchor_idx, anchor_hist, histograms):
        similarities = [cv2.compareHist(anchor_hist, hist, cv2.HISTCMP_CORREL) for hist in histograms]
        similarities[anchor_idx] = -1  # Ignore the anchor image itself
        return np.argmax(similarities)

    def _find_most_dissimilar(self, anchor_idx, anchor_hist, histograms):
        dissimilarities = [cv2.compareHist(anchor_hist, hist, cv2.HISTCMP_CORREL) for hist in histograms]
        dissimilarities[anchor_idx] = 1  # Ignore the anchor image itself
        return np.argmin(dissimilarities)

# Loading the Dataset

In [8]:
# Path to the dataset
rawDataPath = '../../Data/Furniture_Data/'
train_val_test_split = [0.8, 0.15, 0.05]

In [9]:
# Load DataFrame
furniture_dataset = load(rawDataPath)

Error processing ../../Data/Furniture_Data/lamps\Modern\11286modern-lighting.jpg: [Errno 13] Permission denied: '../../Data/Furniture_Data/lamps\\Modern\\11286modern-lighting.jpg'


In [10]:
furniture_dataset

,ImgPath,FileType,Width,Height,Ratio,Mode,Bands,Transparency,Animated,Category,Interior_Style
0,beds\Asian\19726asian-daybeds.jpg,jpg,350,350,1.0,RGB,R G B,False,False,beds,Asian
1,beds\Asian\20027asian-canopy-beds.jpg,jpg,350,350,1.0,RGB,R G B,False,False,beds,Asian
2,beds\Asian\20109asian-panel-beds.jpg,jpg,350,350,1.0,RGB,R G B,False,False,beds,Asian
3,beds\Asian\20508asian-platform-beds.jpg,jpg,350,350,1.0,RGB,R G B,False,False,beds,Asian
4,beds\Asian\20750asian-comforters-and-comforter...,jpg,350,350,1.0,RGB,R G B,False,False,beds,Asian
...,...,...,...,...,...,...,...,...,...,...,...
90078,tables\Victorian\5victorian-side-tables-and-en...,jpg,350,350,1.0,RGB,R G B,False,False,tables,Victorian
90079,tables\Victorian\6victorian-side-tables-and-en...,jpg,350,350,1.0,RGB,R G B,False,False,tables,Victorian
90080,tables\Victorian\7victorian-side-tables-and-en...,jpg,350,350,1.0,RGB,R G B,False,False,tables,Victorian
90081,tables\Victorian\8victorian-dining-tables.jpg,jpg,350,350,1.0,RGB,R G B,False,False,tables,Victorian


In [11]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((120, 120)),
    torchvision.transforms.RandomCrop((110, 110)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Create the pair dataset
pair_dataset = FashionPairDataset(df=furniture_dataset, 
                                  root_dir=rawDataPath, 
                                  transform=transform)

In [12]:
pair_dataloader = DataLoader(pair_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [13]:
# Check one batch
for (imageA, imageB), label in pair_dataloader:
    print(imageA.shape, imageB.shape, label.shape)
    break

torch.Size([256, 3, 110, 110]) torch.Size([256, 3, 110, 110]) torch.Size([256])


# Model Definition

In [14]:
# Define the Siamese Network using PyTorch
class SiameseNetwork(nn.Module):
    def __init__(self, input_shape, embedding_dim=48):
        super(SiameseNetwork, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=2, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.3)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=2, padding=1)
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(64, embedding_dim)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.dropout(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = self.global_avg_pool(x)
        x = x.view(-1, 64)
        x = self.fc(x)
        return x

# Euclidean distance layer
class EuclideanDistance(nn.Module):
    def forward(self, featsA, featsB):
        return F.pairwise_distance(featsA, featsB)

# Define the complete Siamese Network model
class SiameseModel(nn.Module):
    def __init__(self, input_shape, embedding_dim=48):
        super(SiameseModel, self).__init__()
        self.feature_extractor = SiameseNetwork(input_shape, embedding_dim)
        self.euclidean_distance = EuclideanDistance()
        self.fc = nn.Linear(1, 1)

    def forward(self, inputA, inputB):
        featsA = self.feature_extractor(inputA)
        featsB = self.feature_extractor(inputB)
        distance = self.euclidean_distance(featsA, featsB)
        output = torch.sigmoid(self.fc(distance))
        return output

In [15]:
# Initialize model, loss function, and optimizer
model = SiameseModel(IMG_SHAPE).to(DEVICE)
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), momentum=0.9, lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       factor=0.1,
                                                       mode='max',
                                                       verbose=True)

In [ ]:
# Training and testing loop
for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    total_batches = len(pair_dataloader)
    for batch_idx, ((imageA, imageB), labels) in enumerate(pair_dataloader):
        imageA, imageB, labels = imageA.cuda(), imageB.cuda(), labels.float().cuda()
        optimizer.zero_grad()
        outputs = model(imageA, imageB)
        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
